# TP: Régression logistique et `scikit-learn` 



Dans ce premier TP, nous mettons en oeuvre un modèle de régression logistique pour la prédiction de labels binaires $y \in \{0,1\}$. On rappelle que la régression logistique repose sur une modélisation probabiliste, et donc plutôt que de prédire simplement 0 ou 1, on construit un estimateur $\hat P(y=1|x)$ de la probabilité $\mathbf{P}(y=1|x)$ et on définit un classifieur comme
$$\hat y (x_{new}) = \begin{cases} 1 & \text{si } \hat P(y=1|x_{new}) \geq t, \\ 0 & \text{sinon}\end{cases}$$
pour un seuil $t\in(0,1)$ donné. Par défaut, si les labels jouent un rôle symmetrique, $t=1/2$.

## Le jeu de données

Les données utilisées dans ce TP proviennent d'un jeu de données classique qui s'appelle *Census Income dataset* (également connu sous le nom de *Adult Income* et disponible sur le site [https://archive.ics.uci.edu/ml/datasets/adult](https://archive.ics.uci.edu/ml/datasets/adult)).

Ce dataset a été extrait par Barry Becker du recensement américain de 1994 et est souvent utilisé pour des tâches de classification binaire, où l'objectif est de **prédire si un individu gagne plus ou moins de $50\,000$ dollars par an**, en se basant sur les caractéristiques démographiques et professionnelles disponibles. Les **caractéristiques sont qualitatives et quantitatives** et incluent l'âge, le niveau d'éducation, le statut matrimonial, la profession, le pays d'origine, le sexe, le capital investi, le nombre d'heures de travail par semaine, etc.

Le fichier `data/adult_clean.csv` a été obtenu en nettoyant les fichiers publics (voir le notebook `day1_preparation_data.ipynb`): on a retiré les données manquantes et les doublons. 

On utilise le module `pandas` qui est une bibliothèque open-source populaire en Python, largement utilisée pour la manipulation et l'analyse des données. Ce module fournit des structures de données flexibles et performantes: 
- les **DataFrames** qui sont similaires aux tables de base de données ou aux feuilles de calcul Excel
- les **Series** qui sont des tableaux unidimensionnels avec étiquettes.

In [ ]:
import numpy as np
import pandas as pd

### EX 1 : Chargement 

- Importer les données du fichier `data/adult_clean.csv` sous forme d'un `DataFrame` pandas que l'on nomme `dataset`. 
- Visualiser les premières lignes de `dataset`, noter le nombre de caractéristiques (et leurs noms) et le nom de la variable à prédire.
- Quel est le nombre total de données (de lignes) du `dataset`, et la proportion de données `<=50K` et `>50K`?

### EX 2 : Exploration, visualisation 

Familiarisez-vous avec les données.

- Calculer quelques statistiques descriptives (à l'aide de `shape`, `dtypes`, `describe`, `value_counts`).
- Visualiser quelques caractéristiques en fonction des 2 groupes `<=50K` et `>50K`.

Pour tracer des graphiques on pourra utiliser les librairies `matplotlib` et `seaborn` :

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme()

On pourra utiliser les fonctions suivantes :
- `sns.scatterplot` pour des nuages des points
- `sns.histplot` pour des histogrammes et des diagrammes en bâtons

**Exemple** :

In [ ]:
ax = sns.histplot(data=dataset, x="sex", hue="class", 
                  multiple="stack", alpha=0.5, discrete=True)

### EX 3 : Pré-traitement des données

On ne travaille que très rarement sur des donnnées brutes. 

Pour des  **variables quantitatives**, il est important de les  **standardiser** (centrer et réduire) afin de les ramener à  des échelles comparables. Cela diminue également les problèmes numériques. 

Pour une **variable qualitative** ou **catégorielle**, il n'y a en général pas  d'ordre naturel de ses modalités. Donc il ne faut pas bêtement remplacer des catégories *A*, *B*, *C*,... par 1, 2, 3,... Le problème est que dans un modèle de régression, on va calculer par exemple des moyennes sur ces valeurs, mais en général *B* n'est pas la moyenne de *A* et *C* (alors que c'est le cas pour la valeur 2 par rapport à 1 et 3). Il nous faut alors un encodage qui est invariant à l'ordre des modalités. On utilise  le **one-hot encoding** qui consiste à transformer une variable qualitative en plusieurs variables binaires dites **dummies**. Une variable avec K modalités est transformée en K-1 variables binaires. Chacune de ces variables binaires indique pour une catégorie spécifiée si la variable observée est égale à cette catégorie ou pas. Il suffit de K-1 variables binaires pour encoder K catégories, car l'information sur la K-ième catégorie peut être déduite des K-1 autres variables binaires. Donc, si on utilisait K variables binaires, on introduirait des corrélations entre les colonnes, ce qu'il faut éviter dans un cadre de régression.

- Séparer les données en 3 dataframes qu'on nomme :

    - `quantitative`: dataframe avec toutes les covariables quantitatives
    - `qualitative`: dataframe avec toutes les covariables qualitatives
    - `category` : pour les labels observés `class`

En vue de la régression logistique on effectue le pré-traitement suivant: définir les variables 
- `labels` en remplaçant les classes `<=50K` et `>50K` par les valeurs numériques 0 et 1
- `quantitative_norm` obtenu en renormalisant le dataset `quantitative`
- `qualitative_enc` obtenu par l'appel de `get_dummies` du module `pandas` sur le dataset `qualitative` 

## Echantillons : apprentissage et test 

On rappelle que la modélisation se fait en trois temps :
- on sépare les données : TRAIN / TEST
- on apprend le modèle sur TRAIN
- on évalue la performance du modèle appris sur TEST

On coupe les données de façon aléatoire en deux groupes. 
Le plus souvent, on les sépare en 80% pour l'apprentissage et 20% pour le test. D'autres pourcentages courants sont 67-33 ou 50-50.
On   utilise la  fonction `train_test_split` du package `sklearn.model_selection` pour séparer aléatoirement les données. 

Même si le split est aléatoire, on souhaite que les deux échantillons soient tous les deux représentatifs du problème. En particulier, on voudrait qu'ils contiennent le même pourcentage de labels 0 et 1, ce qui est notamment important quand  les labels ne sont pas équilibrés (pas 50-50).

Ici, la proportion de 1 (individus qui sont dans la catégorie `>50K`) est d'environ 0.25 (le vérifier!).

Si le taux n'est pas très élevé, une coupe totalement aléatoire des données risque de produire des sous-échantillons où l'un des deux ne contient que peu de labels qui valent 1. Cela peut être problématique pour l'apprentissage du modèle comme pour l'évaluation de la méthode.

En pratique, dans des problèmes de classification, on force alors la même répartition des labels dans les deux échantillons TRAIN et TEST, via l'option `stratify`.

### EX 4 : Création des échantillons 

Utiliser la fonction `train_test_split` du module `scikit-learn` pour créer à partir de nos données `quantitative_norm`, `qualitative_enc` et `labels` les 4 variables suivantes: 
- `x_train` et `y_train` qui contiennent 80% du jeu de données pour l'apprentissage du modèle de régression logistique 
- `x_test` et `y_test` qui contiennent les 20% restants pour le test.

## Régression logistique

Le but de ce TP est de fitter un modèle de régression logistique à ces données afin de prédire le label (0 pour `<=50K` ou 1 pour `>50K`) pour des  nouveaux individus à partir de leurs caractéristiques. Nous commençons par le modèle simple qui prend en compte toutes les variables.

### EX 5 : Création du modèle et estimation

- Utiliser la fonction `LogisticRegression` de `sklearn.linear_model` pour effectuer une régression logistique simple (i.e. sans pénalisation). Fitter le modèle sur les données `train`. 
- Visualiser les coefficients estimés (intercept et variables).

### EX 6 : Prédiction

La méthode `predict_proba` renvoie 2 colonnes: la première est la probabilité, conditionnellement aux caractéristiques de chaque individu, d'être `<50K` (label 0, négatif) et la seconde la probabilité, conditionnellement aux caractéristiques de chaque individu, d'être `>=50K` (label 1, positif). A partir de ces probas on peut "décider" en fixant un seuil $t \in [0,1]$ d'être ou non dans une des 2 catégories en utilisant le classifieur construit à partir de ces probabilités : 
$$\hat y (x_{new}) = \begin{cases} 1 & \text{si } \hat P(y=1|x_{new}) \geq t, \\ 0 & \text{sinon}\end{cases}$$

- Sur le jeu `test`, calculer $\hat P(y=1|x)$, les probabilités que `>50K` pour chaque individu $x$ (`predict_proba()`) et les prédictions $\hat y(x)$ (`predict()`) que l'on obtient par seuillage des probabilités au seuil de $t = 1/2$.
- Comparer le nombre moyen de positifs prédits par rapport à celui connu dans `y_test`. 
- Visualiser la distribution des probabilités prédites.
- Faire varier le seuil $t \in [0,1]$.

## Mesures de qualité du classifieur 

Le module `metrics` de `scikit-learn` contient plusieurs fonctions qui permettent d'évaluer la qualité des prédictions d'un modèle. Ces métriques sont détaillées dans les sections sur les métriques de classification, les métriques de classement multi-label, les métriques de régression et les métriques de clustering.

In [ ]:
import sklearn.metrics as metrics
print(dir(metrics))

### Erreurs de classification 
Pour chaque individu $i$ de caractéristiques $x_i$ dans l'échantillon, nous avons : 
- un vrai label/valeur $y_i$
- un label/valeur prédit $\hat y_i$

Deux type d'erreurs peuvent se produire :
1. Prédire $\hat y_i =1$ alors que $y_i=0$ (faux positif/false positive, FP)
2. Prédire $\hat y_i =0$ alors que $y_i=1$ (faux négatif, false negative, FN)

Dans un contexte pratique, ces deux erreurs ne sont pas symétriques, parce qu'une erreur peut porter à des  effets plus négatifs que l'autre. En général, les labels sont choisis de sorte à être moins tolérants aux faux positifs qu'aux faux négatifs, pensons par exemple aux tests de grossesse, ou aux procès en justice. 


### EX 7 : Accuracy score  

Le taux de précision (_accuracy score_) est la mesure de performance qui représente la proportion d'observations correctement classées par le modèle parmi l'ensemble total des observations. Ce taux est un pourcentage, où une valeur de 100% signifie que toutes les prédictions du modèle sont correctes.

- Implémenter à la main (en `numpy`) le calcul de ce score pour les données de test.
- Obtenir ce score via l'appel de `accuracy_score` du module `sklearn.metrics` et via la méthode `score` de l'objet `model`.
- Comparer ce score à un estimateur trivial qui consiste à toujours renvoyer la classe la plus fréquente.

### Matrice de confusion, recall, precision

La matrice de confusion permet de visualiser les performances du modèle en termes de prédictions correctes (vrais positifs et vrais négatifs) et d'erreurs de prédiction (faux positifs et faux négatifs). La représentation classique se fait sous forme d'une matrice où les nombres sur la diagonale sont liés aux prédictions correctes, tandis que les nombres hors de la diagonale sont liés aux prédictions incorrectes. 

![](img/confusion-matrix-python.jpg)

Plus précisément: 

- le coin inférieur droit représente les **vrais positifs** (TP): les personnes de la classe `>50K` prédites comme telles par le modèle 
- le coin supérieur gauche représente les **vrais négatifs** (TN): les personnes de la classe `<=50K` prédites comme telles par le modèle 
- le coin supérieur droit représente les **faux positifs** (FP): les personnes de la classe `<=50K` prédites comme étant `>50K`
- le coin inférieur gauche représente les **faux négatifs** (FN): les personnes de la classe `>50K` prédites comme étant `<=50K`

**Attention**:
Dans la page wikipedia la matrice est "inversée": les vrais positifs sont en haut à gauche... 

A partir de ces données on peut construire plusieurs indicateurs de performance, dont le recall et la précision.

#### Recall ou sensitivité

$$\frac{\text{TP}}{\text{\# (real P)}} = \frac{\text{TP}}{\text{TP+FN}}$$
Mesure la capacité du classifieur à identifier tous les exemples positifs réels, c'est-à-dire sa sensibilité ou son taux de détection. Dans notre cadre, le recall est le taux d'individus identifiés comme `>50K` par le classifeur parmi tous les individus avec `>50K` dans `y_test`.

#### Precision
$$\frac{\text{TP}}{\text{\# (predicted P)}} = \frac{\text{TP}}{\text{TP+FP}}$$
Mesure la capacité du classifieur à ne prédire positif que lorsque la prédiction est vraiment correcte.  P. ex.  le classifieur qui prédit systématiquement `>50K` pour tous les individus a un  recall  de 100%, mais sa precision est de 0%. Donc, on veut que les deux scores, le recall et la precision, d'un classifieur soient élevés.



### EX 8


- Tracer la matrice de confusion en utilisant `ConfusionMatrixDisplay` et donner les scores `recall` et `precision`.

### Courbe ROC, AUC

Les valeurs prédites `y_predicted` ci-dessus ont été obtenues en seuillant les probabilités de `>50K` calculées par le modèle de régression logistique avec le seuil par defaut $t=1/2$.

$$\hat y_t (x_{new}) = \begin{cases} 1 & \mathbf{\hat P}(y=1 | x_{new}) \geq t, \\ 0 & \text{sinon.}\end{cases}$$

Or, en variant le seuil $t\in(0,1)$ il est possible que le classifieur associé ait des meilleurs scores.

La courbe ROC (Receiver Operating Characteristic) est une courbe paramétrique, paramétrée par $t \in [0,1]$. Elle représente les points $(\text{FPR}_t, \text{TPR}_t)$ pour tout seuil $t\in[0,1]$, où 

- $\text{TPR}_t$ (True positive rate) est le recall en fonction du seuil $t$ :
$$\frac{\text{TP}_t}{\text{TP}_t+\text{FN}_t}$$

- $\text{FPR}_t$ (False positive rate) est définit par la ratio suivant en fonction du seuil $t$ :
$$\frac{\text{FP}_t}{\text{FP}_t+\text{TN}_t}$$

Avec le seuil $t=1$, le classifieur prédit `<=50K` systématiquement, donc il n'y a pas de positifs et on a $\text{TP}_0 = 0$ et $\text{FP}_0 = 0$. En revanche, avec un seuil de $t=0$, le classifieur prédit toujours `>50K` et donc on a $\text{TP}_1 = 1$ et $\text{FP}_1 = 1$. Toutes les courbes ROC commencent donc au point $(1, 1)$ et se terminent au point $(0,0)$. La fonction $y=x$ represente la courbe ROC de la famille di classifieurs $\hat y_t (x_{new}) = 1$ avec probabilité $t\in(0,1)$.

Si la courbe est élevée, le modèle a une très bonne performance quelque soit le seuil. Cela est mesuré par l'aire sous la courbe (AUC -- Area Under the ROC curve). L'AUC permet de comparer deux modèles, ou un modèle sur des jeux de données différentes, comme les données train et test.

### EX 9
- Utiliser la fonction `RocCurveDisplay` pour tracer la courbe ROC de notre modèle pour les données train et les données test. Sur quelles données observe-t-on une meilleure performance ?
- Utiliser la fonction `roc_auc_score` pour calculer l'AUC sur les données de test et vérifier qu'elle correspond à l'AUC indiquée sur le tracé de courbe au point précédent.

### Courbe precision/recall, AP

Un autre outil de comparaison est la courbe precision/recall qui représente les points des coordonnées (recall, precision) pour tout seuil $t\in[0,1]$. L'interprétation est la même : plus la courbe est élevée (ou plus l'aire sous la courbe est grande), mieux c'est.

Noter que contrairement à la courbe ROC, la courbe precision/recall n'est pas toujours monotone.

### EX 10
- Tracer la courbe precision/recall de notre modèle sur les données test en utilisant la fonction `PrecisionRecallDisplay`.


## Régression logistique avec pénalisations

### Régression ridge

Nous allons utiliser une pénalité en norme 2 (dite $\ell_2$ ou Ridge) sur les coefficients de la régression. Cette pénalité Ridge a tendance à "rétrecir" les valeurs des coefficients (on parle de shrinkage) et régularise la solution.

### EX 11

- Mettre en oeuvre la régression logistique avec pénalité $\ell_2$ sur les données d'apprentissage. On utilisera toujours la fonction `LogisticRegression`avec  l'option `penalty='l2'`. On utilisera une pénalité assez forte, p. ex. `C=0.01`. 
- Comparer les valeurs des coefficients avec le cas sans pénalité (visualisez l'effet de shrinkage). 
- Évaluer les performances du classifieur sur les données de test.
- Pour les deux classifieurs (avec et sans pénalisation $\ell_2$), comparer les courbes de score sur les données `test` ainsi que les performances en termes de courbes (ROC ou precision-recall) et d'AUC. 

### Régression lasso

Dans cette partie, on va mettre en oeuvre une pénalité $\ell_1$ dans la régression logistique. Cela peut permettre de faire de la sélection de variables : les variables peu informatives pour la prédiction ne seront plus du tout utilisées (coefficient $w_i$ estimé à 0). On peut ainsi construire un estimateur plus parcimonieux. 

### EX 12

- Mettre en oeuvre la régression logistique avec pénalité $\ell_1$ sur les données d'apprentissage (avec l'option le `solver='liblinear'`).
- Comparer les valeurs des coefficients avec le cas sans pénalité et avec pénalité $\ell_2$ (visualiser l'effet d'annulation des coefficients). Tester différentes valeurs de $C$.
- Pour l'ensemble des classifieurs (avec et sans pénalisation $\ell_1$ et $\ell_2$), comparer les performances en termes de courbes (ROC et precision-recall) et d'AUC. 

### Chemins de régularisation pour le Lasso

Le but est de visualiser l'effet de la **constante de pénalisation** sur l'évolution des coefficients $w_i$, à travers les **chemins de régularisation**. Lorsque la pénalité est très forte (i.e $C$ très petite), aucune  variable n'est sélectionnée (tous les coefficients $w_i$ sont estimés à 0). Dans ce cas, la fonction de régression logistique est constante (on a seulement l'intercept) et le lien entre $y$ et $X$ est très mal appris. Puis au fur et à mesure que $C$ augmente, on inclue de plus en plus de variables dans notre modèle de régression logistique. Lorsque $C$ est très grande, on ne pénalise plus et on retrouve les résultats de la régression logistique simple. 

### EX 13

- Fixer une grille de valeurs de $C$, estimer le modèle lasso pour chaque constante $C$ et afficher les coefficients en fonction de $C$. 

## Validation croisée

La validation croisée consiste à entrainer le modèle sur différents sous-échantillons des données et d'évaluer à chaque fois sa capacité prédictive sur le reste des données. 
La validation croisée permet de choisir la meilleure constante $C$ de la pénalité lasso ou Ridge. Plus généralement, dans de nombreux autres modèles, la validation croisée est utilisée pour ajuster les hyperparamètres du modèle de façon optimale en évitant le surapprentissage et augmentant sa capacité de généralisation sur des nouvelles données.

### EX 14 :  Stabilité des prédictions

- Utiliser la fonction `cross_validate` du module `sklearn.model_selection` pour obtenir un intervale de confiance de l'accuracy score. On fera cette étude sur le modèle de régression logistique simple `model` et celui avec pénalité $\ell_1$ `model_lasso`.

### Choix de la constante de pénalisation

On peut aussi utiliser la validation croisée pour choisir un hyperparamètre. Dans cette partie, nous allons mettre en oeuvre le choix de la constante de pénalisation par **validation croisée**. 
Nous le ferons dans le cadre d'une pénalité $\ell_1$, mais on pourrait faire exactement la même chose avec un autre type de pénalité. 

Par défaut, la fonction `LogisticRegression` ne sait pas choisir la constante de pénalisation automatiquement et par défaut cette constante est fixée à 1. Ce choix n'a pas de justification et n'a donc aucune raison d'être utilisé. 

### EX 15

- Utiliser la fonction `LogisticRegressionCV` pour choisir la constante $C$ qui maximise le score AUC (aire sous la courbe ROC) dans une grille de valeur de $C$ fixée.

## `scikit-learn` et automatisation: `pipeline`

En `scikit-learn`, un _pipeline_ est une séquence ordonnée d'étapes de prétraitement des données et de modélisation regroupées en une seule entité. Il permet de définir et d'automatiser un flux de travail cohérent pour le traitement des données et l'entraînement d'un modèle.

Il se compose en général de plusieurs étapes:
- transformations des données :  normalisation des variables, l'imputation des valeurs manquantes, la réduction de dimension, etc. Elles permettent de préparer les données avant de les fournir au modèle.
- le modèle d'apprentissage automatique.
- la validation croisée : pour évaluer les performances du modèle.

L'avantage d'utiliser un pipeline est qu'il permet de regrouper toutes ces étapes en une seule entité. 

### EX 16 : Création d'un pipeline pour notre régression logistique 

- Définir à partir des classes `OneHotEncoder`, `StandardScaler` de `sklearn.preprocessing` et `ColumnTransformer` de `sklearn.compose` un préprocesseur qui réalise les mêmes transformations que celles faites dans la section 1.1.5 (Pré-traitement des données). Attention, la transformation `OneHotEncoder` doit s'appliquer uniquement aux données qualitatives et la transformation `StandardScaler` doit s'appliquer uniquement aux données quantitatives. 
- Créer un pipeline avec la fonction `make_pipeline` de `sklearn.pipeline` pour définir un modèle de régression logistique avec prétraitement des données automatique (fait par le préprocesseur).

### EX 17 : Utilisation du pipeline 

- Faire une estimation du modèle. Attention, quelles données doivent être utilisées ?
- Faire des prédictions et calculer les différents scores sur le jeu de test.
- Faire une étape de validation croisée. 

### EX 18 : Avec une autre méthode de classification : Gradient Boosting

L'avantage d'utiliser un module riche comme `scikit-learn` est qu'on peut facilement changer une brique pour tester d'autres modèles. Ici on propose de remplacer la brique `LogisticRegression` par une méthode basée sur les arbres de décisions. Les arbres de décisions et les méthodes de "boosting" seront expliquées dans le second cours mais voyons les résultats que l'on peut obtenir. 

- Utiliser le préprocesseur `OneHotEncoder` avec l'option `sparse_output` à `False` et la classe `HistGradientBoostingClassifier` du module `sklearn.ensemble` pour composer un pipeline que l'on nomme `model_gradboost`.

- Comparer les courbes ROC et Precision/Recall obtenues avec cet estimateur et la régression logistique. 